In [1]:
import digger as dg
import os
import json
import pandas as pd
import numpy
from  datetime import date
from  datetime import time
from  datetime import datetime
from  datetime import timedelta
from dateutil.relativedelta import relativedelta
from io import BytesIO
from IPython.display import HTML, display
from ipywidgets import widgets
from random import getrandbits
from pathlib import Path

#Считываем локальный json файл с учетными данными разных БД

def display_error(err):
    """
    Красненькое сообщение об ошибке
    """
    with out:
        display(HTML(f'<br><div style="background: rgba(255,100, 100, 0.1); padding: 10px;">{err}</div>'))     

def set_dg_auth():
    """
    Проверяет что пользователь правильно авторизовался в диггере
    """
    dg.set_auth(login_user.value, login_password.value)
    try:
        dg.get_tables()
    except Exception as ex:
        display_error(ex)
        raise(ex)

def check_common_filled(common):
        """
        Проверяет заполненность разделов
        """
        for widget in common.children:
            if isinstance(widget, widgets.Checkbox):
                if widget.value:
                    return True
            elif isinstance(widget, widgets.Text):
                if widget.value.strip():
                    return True
            elif isinstance(widget, widgets.Dropdown):
                if widget.value and widget.value != 'Не выбрано':
                    return True
            elif isinstance(widget, widgets.VBox) or isinstance(widget, widgets.HBox):
                if check_common_filled(widget):
                    return True
        return False
    
def check_data(start_date, end_date, user_id, UTM_source, UTM_campaign, common,push_campaign_ids ):
    """
    Разные проверки на правильность введенных данных
    """
    err=''
    #проверка дат
    if pd.isnull(start_date.value) or  pd.isnull(end_date.value):
        err = 'Необходимо указать период. Проверьте введённые даты'
        display_error(err)
        raise(ValueError(err))
    elif start_date.value > end_date.value:
        err = 'Начало периода не может быть больше конца периода. Проверьте введённые даты'
        display_error(err)
        raise(ValueError(err))
    elif start_date.value > date.today() or end_date.value > date.today() :
        err='Введена дата, которая еще не наступила. Проверьте введённые даты'
        display_error(err)
        raise(ValueError(err))
    elif end_date.value>= start_date.value+relativedelta(months=12):
        err='Выбран слишком большой период. Разбейте на несколько периодов'
        display_error(err)
        raise(ValueError(err))

        
    def validate_selection():
        # Проверка для раздела "Друзья конкретного пользователя"
        if user_id.value.strip():
            return True
        
        # Проверка для раздела "Перешедшие по ссылке"
        if UTM_source.value.strip() or UTM_campaign.value.strip():
            return True
        # Проверка для раздела "Перешедшие по пушу"
        if push_campaign_ids.value.strip():
            return True
        
        # Проверка для раздела "Общие выгрузки"
        if check_common_filled(common):
            return True
        
        # Если ничего не выбрано
        return False
    if not validate_selection():
        err = "Пожалуйста, сделайте выбор в одном из разделов."
        display_error(err)
        raise(ValueError(err))

# def get_data(user_id, min_value_text, max_value_text):
#     cnt_friends = '' 
#     if user_id.value.strip():
#         if ((min_value_text.value.strip() == None or min_value_text.value.strip().int() == 0) 
#                  and  
#                  (max_value_text.value.strip() == None or max_value_text.value.strip().int() == 0)
#            ):
#             cnt_friends = ''  
#         else:
#             if min_value_text.value.strip() == None:
#                 min = 0
#             else:
#                 min = min_value_text.value.strip().int()
#             if max_value_text.value.strip() == None:
#                 max = None
#             else:
#                 max = max_value_text.value.strip().int() 
#             if max == None:
#                 cnt_friends =f'AND c.cnt_friends >= {min}'
#             else:
#                 cnt_friends =f'AND c.cnt_friends >= {min} and c.cnt_friends <={max}'
    
#     with open('query_user_friends.sql', encoding='utf-8') as f:
#          query = f.read().format(user_id =user_id.value.strip()
#                                          , cnt_friends =cnt_friends 
#                                         )
                
#     return query    
def get_data(user_id, min_value_text, max_value_text,IsNew, project_options, started_onboarding,finished_onboarding_options, not_finished_onboarding_options, no_geo, UTM_source, UTM_campaign, start, end, not_allowed,push_campaign_ids, event_name ):
    
    #собираем данные по кол-ву друзей
    cnt_friends = '' 
    
    min_value = min_value_text.value.strip()
    max_value = max_value_text.value.strip()
    if not min_value:
        min_value = 0
    else:
        min_value = int(min_value)
        
    if not max_value:
        max_value = None
    else:
        max_value = int(max_value)
    
    # Формирование cnt_friends в зависимости от значений min и max
    if max_value is None and (min_value is not None and min_value != 0):
        cnt_friends = f'AND c.cnt_friends >= {min_value}'
    elif max_value is None and (min_value is None or min_value ==0):
         cnt_friends = ''
    elif max_value == 0 and min_value ==0:
         cnt_friends = f'AND c.user_id IS NULL'
    elif max_value != 0 and min_value ==0:
         cnt_friends = f'AND (c.user_id IS NULL or ( c.cnt_friends >= {min_value} and c.cnt_friends <= {max_value}))'
    else:
        cnt_friends = f'AND c.cnt_friends >= {min_value} and c.cnt_friends <= {max_value}'

    #собираем данные по первому другу
    is_new = ''
    if IsNew.value:
        is_new = 'AND f.n = 1 ---является первым другом'
    else:
        is_new = ''
    
    #собираем данные по проектам
    if project_options.value in [
        'Не указывать проект'
        ]:
        projects_txt = ''
    else:
        if project_options.value == 'Искать по родному проекту пользователя':
            projects_txt = f'AND a.nativeproject IN ({project_id_text.value})'
        elif project_options.value == 'Искать по родной стране пользователя':
            projects_txt = f'AND a.nativeproject IN (SELECT distinct id FROM projects WHERE country IN({project_id_text.value}))'
        elif project_options.value == 'Искать по родной стране, а в случае отсутствия - по последней запущенной стране':
            projects_txt = f'AND nvl(a.nativeproject,a.lastLaunchSegment) IN (SELECT distinct id FROM projects WHERE country IN({project_id_text.value}))'
        else:
            projects_txt = f'AND nvl(a.nativeproject,a.lastLaunchSegment) IN ({project_id_text.value})'
    
    #собираем данные по онбордингу
    onboarding_start_subquery = ''
    onboarding_start_join = ''
    if started_onboarding.value: 
        onboarding_start_subquery = '''
        , onboarding_start_subquery
            --- начинал онбординг (видел слайд кроме последнего или тапнул Добавить друзей на экране дашборда)
            (
              select distinct
                     user,
                     authuser,
                     appinstanceid,
                     receivets
              from
              (
              select distinct
              common.user
              , common.authuser
              , common.appinstanceid
              , common.sessionId
              , receivetimestamp as receivets
              from view
              where nvl(fraud, 0) = 0
                and uiElementName IN ('friendsTipsOpeningSlide','friendsTipsBackgroundGeoSlide','friendsTipsNoGeoSlide','friendsTipsAllowGeoSlide','friendsTipsAuthSlide','friendsTipsPhysicalActivitySlide',
                                    'friendsTipsEndingSlide','friendsTipsDisabledPushSlide','friendsTipsPushSlide','friendsTipsLocationPushSlide')
                and uiElementOwnerName = 'friendsTipsScreen'
              
              UNION ALL
              
              select distinct
              common.user
              , common.authuser
              , common.appinstanceid
              , common.sessionId
              , receivetimestamp as receivets
              from  action 
              where nvl(fraud, 0) = 0
                and actionType = 'tap'
                and uiElementName = 'addFriendsPromo'
                and uiElementOwnerName = 'dashboardFriends'
              )  
            )
            '''
        onboarding_start_join = 'JOIN onboarding_start_subquery os ON a.appinstanceid = os.appinstanceid'

    onboarding_finished_subquery = ''
    onboarding_condition = ''
    onboarding_finished_join = ''
    if finished_onboarding_options.value != 'Не выбрано' or not_finished_onboarding_options.value  != 'Не выбрано':
        if finished_onboarding_options.value == 'В указанный период' or not_finished_onboarding_options.value == 'В указанный период' :
            onboarding_finished_subquery = '''
            , onboarding_finished_subquery
                ---прошел онбординг
                (
                  select
                    distinct
                        t.user, receivets
                  from first_launch AS t
                  where
                    t.event = 'friends_on_map'
                    and t.prod IN (24,25,26)
                    union
      
              select distinct 
                common.user,
                receiveTimestamp
                from view
                where
                  common.user is not null and
                  nvl(fraud, 0) = 0 
                    AND (
                      (uiElementName = 'dashboardFriends'  AND userSettings.friendsOnMapEnabled = true)
                  OR uiElementName = 'friendsTipsEndingSlide' 
                      )
            
                union
                select distinct 
                common.user,
                receiveTimestamp
                from action
                where
                  common.user is not null and
                  nvl(fraud, 0) = 0 and
                    uiElementName IN ('friendsTipsScreen')   
                    and actiontype = 'close'
                    AND userSettings.friendsOnMapEnabled = true
                    )
        '''
        else:
            onboarding_finished_subquery = '''
            , onboarding_finished_subquery
                ---прошел онбординг
                (
                  select
                    distinct
                        t.user, receivets
                  from first_launch AS t
                  where
                    t.event = 'friends_on_map'
                    and t.prod IN (24,25,26)  
                )
        '''
        if not_finished_onboarding_options.value == 'Вообще никогда' or finished_onboarding_options.value == 'Когда-либо' :
            onboarding_finished_join = 'LEFT JOIN onboarding_finished_subquery of ON a.user = of.user'
            if not_finished_onboarding_options.value == 'Вообще никогда':
                onboarding_condition = 'AND of.user IS NULL' 
            if finished_onboarding_options.value == 'Когда-либо':
                onboarding_condition = 'AND of.user IS NOT NULL' 
        elif not_finished_onboarding_options.value == 'В указанный период' or finished_onboarding_options.value == 'В указанный период' :
            onboarding_finished_join = f'''
                                    LEFT JOIN onboarding_finished_subquery of 
                                        ON a.user = of.user 
                                        AND date_format(from_unixtime(of.receivets/1000), 'yyyy-MM-dd') >= date_format({ f"'{start.value.strftime('%Y-%m-%d')}'"}, 'yyyy-MM-dd')
                                        AND date_format(from_unixtime(of.receivets/1000), 'yyyy-MM-dd') <= date_format({ f"'{end.value.strftime('%Y-%m-%d')}'"}, 'yyyy-MM-dd')
                                        '''
            if not_finished_onboarding_options.value == 'В указанный период':
                onboarding_condition = 'AND of.user IS NULL' 
            if finished_onboarding_options.value == 'В указанный период':
                onboarding_condition = 'AND of.user IS NOT NULL' 
    
    #собираем данные по геопермиссиям
    geo_subquery = ''
    geo_join = ''
    if no_geo.value: 
        geo_subquery = '''
        , no_geo
            ---в последнем запуске нет гео
            (
              select
              common.user
              , common.authuser
              , common.appinstanceid
              , receivetimestamp as receivets
              , userSettings.map.alwaysAllowLocation
              , row_number() over(partition by common.user order by receivetimestamp desc) as n
              from audience_counter
              where nvl(fraud, 0) = 0 and common.product IN (24,25,26)
            )'''
        geo_join = 'JOIN no_geo g ON a.appinstanceid = g.appinstanceid AND g.n = 1 AND alwaysAllowLocation = False' 

    #собираем данные по UTM
    def format_textarea_content(text):
        # Разделяем строку по запятым и удаляем лишние пробелы
        words = [word.strip().lower() for word in text.split(',') if word.strip()]
        # Преобразуем в строку вида "'слово', 'слово еще одно', 'одно'"
        formatted_string = ", ".join(f"'{word}'" for word in words)
        return formatted_string

    utm_condition_installs_source = ''
    utm_condition_installs_campaign = ''
    utm_condition_action_source = ''
    utm_condition_action_campaign = ''
    if UTM_source.value.strip():
        utm_condition_installs_source = f'and lower(source)  IN ({format_textarea_content(UTM_source.value)})'
        utm_condition_action_source = f'and lower(launch.utm.source)  IN ({format_textarea_content(UTM_source.value)})'
    if UTM_campaign.value.strip():
        utm_condition_installs_campaign = f'and lower(campaign)  IN ({format_textarea_content(UTM_campaign.value)})'
        utm_condition_action_campaign = f'and lower(launch.utm.campaign)  IN ({format_textarea_content(UTM_campaign.value)})'

    push_ids = ''
    if push_campaign_ids.value.strip():
        push_ids = format_textarea_content(push_campaign_ids.value)
    
    #собираем данные по ограничениям на пуши
    allowed_condition = ''
    allowed_appinstanceid = ''
    join_type = ''
    if not_allowed.value: 
        allowed_condition = '''
         having
      -- Продакт
      max(pr.authuser is not null)
      -- Либо отвечает необходимым условиям
      or (
        max_by(pa.notRefused, pa.lastLaunchTs) --не запрещено отслеживание в последнем запуске
        and not max(array_contains(pa.launchedCountries,11)) -- не запускал Украину никогда
        and not max(cu.authUser is not null) -- не был в custom_ukraine_auth никогда
      )
      '''
        allowed_appinstanceid = '''if(
        max_by(pa.notificationAllowed, pa.lastLaunchTs),
        max_by(pa.appInstanceId, pa.lastLaunchTs),
        null
      ) as appInstanceId,'''
    else:
        allowed_condition = ''
        allowed_appinstanceid = 'max_by(pa.appinstanceid, pa.lastLaunchTs) as appinstanceid,'
        join_type = 'LEFT'
    
    if user_id.value.strip():
        with open('query_user_friends.sql', encoding='utf-8') as f:
         query = f.read().format(user_id=user_id.value.strip()
                                 , cnt_friends=cnt_friends
                                 , is_new = is_new
                                 , projects_txt = projects_txt
                                , allowed_condition = allowed_condition
                                , allowed_appinstanceid = allowed_appinstanceid
                                 , start = f"'{start.value.strftime('%Y-%m-%d')}'"
                                 , end =  f"'{end.value.strftime('%Y-%m-%d')}'"
                                , join_type = join_type)
    elif UTM_source.value.strip() or UTM_campaign.value.strip():
        with open('query_utms.sql', encoding='utf-8') as f:
         query = f.read().format(cnt_friends=cnt_friends
                                 , projects_txt = projects_txt
                                 , onboarding_start_subquery = onboarding_start_subquery 
                                 , onboarding_start_join = onboarding_start_join
                                 , onboarding_finished_subquery = onboarding_finished_subquery
                                 , onboarding_condition = onboarding_condition
                                 , onboarding_finished_join = onboarding_finished_join
                                 , geo_subquery = geo_subquery
                                 , geo_join = geo_join
                                 , utm_condition_installs_source = utm_condition_installs_source
                                , utm_condition_installs_campaign = utm_condition_installs_campaign
                                , utm_condition_action_source = utm_condition_action_source
                                , utm_condition_action_campaign = utm_condition_action_campaign
                                 , start = f"'{start.value.strftime('%Y-%m-%d')}'"
                                 , end =  f"'{end.value.strftime('%Y-%m-%d')}'"
                                , allowed_condition = allowed_condition
                                , allowed_appinstanceid = allowed_appinstanceid)
    elif  push_campaign_ids.value.strip() :
        with open('query_push_campaigns.sql', encoding='utf-8') as f:
         query = f.read().format(cnt_friends=cnt_friends
                                 , projects_txt = projects_txt
                                 , onboarding_start_subquery = onboarding_start_subquery 
                                 , onboarding_start_join = onboarding_start_join
                                 , onboarding_finished_subquery = onboarding_finished_subquery
                                 , onboarding_condition = onboarding_condition
                                 , onboarding_finished_join = onboarding_finished_join
                                 , geo_subquery = geo_subquery
                                 , geo_join = geo_join
                                 , push_ids = push_ids
                                , event_name = event_name
                                 , start = f"'{start.value.strftime('%Y-%m-%d')}'"
                                 , end =  f"'{end.value.strftime('%Y-%m-%d')}'"
                                , allowed_condition = allowed_condition
                                , allowed_appinstanceid = allowed_appinstanceid)
    elif check_common_filled(common): 
        with open('query_common.sql', encoding='utf-8') as f:
         query = f.read().format(cnt_friends=cnt_friends
                                 , projects_txt = projects_txt
                                 , onboarding_start_subquery = onboarding_start_subquery 
                                 , onboarding_start_join = onboarding_start_join
                                 , onboarding_finished_subquery = onboarding_finished_subquery
                                 , onboarding_condition = onboarding_condition
                                 , onboarding_finished_join = onboarding_finished_join
                                 , geo_subquery = geo_subquery
                                 , geo_join = geo_join
                                , allowed_condition = allowed_condition
                                , allowed_appinstanceid = allowed_appinstanceid
                                , start = f"'{start.value.strftime('%Y-%m-%d')}'"
                                , end =  f"'{end.value.strftime('%Y-%m-%d')}'"
                                )
    
    return query
    


# def get_query(email, authuser, user, ip, start, end, acts):
#     """
#     В зависимости от введенных данных подбирает нужный запрос
#     """
#     #если заполнен емейл то мы сначала ищем идентификатор пользователя, показываем его и ищем и по нему
#     if user_id.value.strip()
#     return query
# 

def post_request(start, end, query):
    """
    Дожидается выполнения запроса диггера
    """
    try:
        with out:
            print('Выполняем запрос (может занять некоторое время):')
            result = dg.post_request(
                query,
                start,
                end,
                name='friends_data_search_voila',
                check_data_exists=True, retries = 1000
            )
            print('Запрос успешно выполнен')
    except Exception as ex:
        display_error(ex)
    # return result



In [2]:
def main(*args, **kwargs):
    """
    Функция которая срабатывает при нажатии кнопки "Сгенерировать файл"
    объединяет в себе все остальные функции
    """
    out.clear_output()
    with link_out:
        link_out.clear_output()
    button.disabled = True
    #авторизуемся в диггере
    try:
        set_dg_auth()
        check_data(start, end, user_id, UTM_source, UTM_campaign, common,push_campaign_ids)
        query = get_data(user_id, min_value_text, max_value_text,IsNew, project_options, started_onboarding,finished_onboarding_options, not_finished_onboarding_options, no_geo, UTM_source, UTM_campaign, start, end, not_allowed,push_campaign_ids, push_status.value.strip())
        if user_id.value.strip():
            res = post_request(date(2023,10,1),end.value,query)
        else:
            res = post_request(start.value,end.value,query)
        
    #     #возвращаем результат запроса
    #     if len(res) == 0:
    #         with out:
    #             print('За искомый период ничего не найдено.')
    #     else:
    #         with out:
    #             print(f'За искомый период найдено {len(res)} событий.')
    #             download_file(res)
                
    #             #outer()
    #             #out.clear_output()
    except Exception as ex:
        display_error(ex)
        button.disabled = False
        #pass
    finally:
        button.disabled = False

<!-- # Поиск событий по определенному пользователю/email/IP
[**Инструкция**](https://confluence.2gis.ru/pages/viewpage.action?pageId=449562179)   

Инструмент предназначен для поиска событий по конкретным идентификаторам пользователей/емейлам/IP-адресам за обозначенные даты.   
Сейчас поиск возможен только по периоду с января 2021 года.
Если нужны более ранние данные, необходимо обратиться  в *~team-casino* для того, чтобы они досчитали данные за нужный период для таблицы **dgis_users_data_full**

В инструменте есть 4 варианта поиска:
* Поиск по идентификатору пользователя **(user)** 
* Поиск по идентификатору авторизованного пользователя **(authuser)** 
* Поиск по **email**
* Поиск по **IP-адресу**

Особенности использования инструмента:
* Логин и пароль используются от вашей виндовой учетки, чтобы отправить от её имени запрос к диггеру
* У вас должен быть доступ к таблице **ugc_auth_user** в диггере. Доступ можно запросить в *~team-casino* -->

# Выгрузки appinstanceid/email/advertisingId по Друзьям на карте

Для получения более подробной информации по использованию инструмента и выбора сценария взаимодействия вы можете ознакомиться со  [**статьей** ](https://confluence.2gis.ru/pages/viewpage.action?pageId=500080708)

* *позволяет выгружать друзей конкретного публичного профиля (поиск при этом производится по user_id (короткий числовой идентификатор), а не по public_user_id)*
* *позволяет выгружать пользователей, перешедших по определенной UTM-метке*
* *позволяет выгружать пользователей, которым был отправлен пуш (notificantionSent) или которые открыли пуш (notificationTapped)*
* *позволяет производить общие выгрузки на основе совершенных действий (старт онбординга, прохождение онбординга, добавление друга) в Друзьях*

Важно! Инструмент выгружает только пользователей, у которых был запуск за последние 12мес.

In [ ]:
login_user = widgets.Text(description='Логин: ', placeholder='i.ivanov', style=dict(description_width='100px'))
login_password = widgets.Password(description='Пароль: ', style=dict(description_width='100px'))
grid1 = widgets.GridspecLayout(2,1, grid_gap='5px', width='650px')

grid1[0,0] = login_user
grid1[1,0] = login_password
grid1

* Логин и пароль используются от вашей виндовой учетки, чтобы отправить от её имени запрос к диггеру
* У вас должен быть доступ к таблице **ugc_auth_user** в диггере. Доступ можно запросить в *~team-casino*

In [4]:
min_value_text = widgets.Text(
    value=None,
    description='От (включительно):',
    placeholder='Не обязательно указывать',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='350px')
)

max_value_text = widgets.Text(
    value=None,
    description='До (включительно):',
    placeholder='Не обязательно указывать',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='350px')
)

friends_count = widgets.HBox([min_value_text, max_value_text])

IsNew = widgets.Checkbox(value=False,description ='Только те, у кого пользователь стал первым другом', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px')) 

# Создаем заголовок и Dropdown для группы "Не закончили онбординг"
not_finished_onboarding_label = widgets.HTML(value="Не закончили онбординг:")
not_finished_onboarding_options = widgets.Dropdown(
    options=[
        'Не выбрано',
        'Вообще никогда',
        'В указанный период'
    ],
    value='Не выбрано',  # По умолчанию ничего не выбрано
    description='',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)


# Создаем заголовок и Dropdown для группы "Закончили онбординг"
finished_onboarding_label = widgets.HTML(value="Закончили онбординг:")
finished_onboarding_options = widgets.Dropdown(
    options=[
        'Не выбрано',
        'Когда-либо',
        'В указанный период'
    ],
    value='Не выбрано',  # По умолчанию ничего не выбрано
    description='',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Функции для активации/деактивации виджетов
def update_not_finished_onboarding_options(change):
    if change['new'] != 'Не выбрано':
        finished_onboarding_options.disabled = True
    else:
        finished_onboarding_options.disabled = False

def update_finished_onboarding_options(change):
    if change['new'] != 'Не выбрано':
        not_finished_onboarding_options.disabled = True
    else:
        not_finished_onboarding_options.disabled = False

# Привязываем функции к изменениям в Dropdown
not_finished_onboarding_options.observe(update_not_finished_onboarding_options, names='value')
finished_onboarding_options.observe(update_finished_onboarding_options, names='value')

# Упаковываем заголовок и Dropdown в вертикальный контейнер VBox
not_finished_onboarding_group = widgets.VBox([not_finished_onboarding_label, not_finished_onboarding_options])
finished_onboarding_group = widgets.VBox([finished_onboarding_label, finished_onboarding_options])

# Чекбоксы для других опций
started_onboarding = widgets.Checkbox(
    value=False,
    description='Начали проходить онбординг в указанный период',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

no_geo = widgets.Checkbox(
    value=False,
    description='Нет постоянного доступа к гео в последнем запуске за указанный период',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Объединяем все виджеты в логические группы
all_onboarding = widgets.HBox([not_finished_onboarding_group, finished_onboarding_group])
actions = widgets.VBox([
    widgets.HTML(value="<b>Старт онбординга:</b>"),
    started_onboarding,
    widgets.HTML(value="<b>Окончание онбординга:</b>"),
    all_onboarding,
    widgets.HTML(value="<b>Другие опции:</b>"),
    no_geo,
    widgets.HTML(value="<b>Кол-во друзей:</b>"),
    friends_count
    
])

user_id = widgets.Text(description='Id пользователя: ', style=dict(description_width='100px'))
user_data = widgets.VBox([user_id, widgets.HTML(value="Кол-во друзей:"), friends_count,IsNew])

UTM_source = widgets.Textarea(description='UTM_source: ', placeholder='если несколько, то через запятую', style=dict(description_width='100px'))
UTM_campaign = widgets.Textarea(description='UTM_campaign: ', placeholder='если несколько, то через запятую', style=dict(description_width='100px'))
UTM_data = widgets.VBox([UTM_source, UTM_campaign,actions])

common = widgets.VBox([
    widgets.HTML(value="<b>Старт онбординга:</b>"),
    started_onboarding,
    widgets.HTML(value="<b>Окончание онбординга:</b>"),
    all_onboarding,
    widgets.HTML(value="<b>Другие опции:</b>"),
    no_geo,
    widgets.HTML(value="<b>Кол-во друзей:</b>"),
    friends_count
    
])

push_campaign_ids = widgets.Textarea(description='ID кампании по пушам:', placeholder='если несколько, то через запятую', style=dict(description_width='150px'),
    layout=widgets.Layout(width='600px'))
push_status = widgets.RadioButtons(
    options=[ 'notificationSent',
        'notificationTapped'
    ],
    description='Событие пуша',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)
push_campaigns_data = widgets.VBox([push_campaign_ids,push_status, actions])

search_data = widgets.Accordion(children=[user_data,UTM_data,push_campaigns_data,common], selected_index=None)
search_data.set_title(0, 'Друзья конкретного пользователя')
search_data.set_title(1, 'Перешедшие по ссылке с определенной UTM')
search_data.set_title(2, 'Пользователи по пушам')
search_data.set_title(3, 'Общие выгрузки')

# Создаем виджет RadioButtons с опциями
project_options = widgets.RadioButtons(
    options=[ 'Не указывать проект',
        'Искать по родному проекту пользователя',
        'Искать по родному проекту, а в случае отсутствия - по последнему запущенному проекту',
        'Искать по родной стране пользователя',
        'Искать по родной стране, а в случае отсутствия - по последней запущенной стране'
    ],
    description='Проект/страна:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Создаем текстовое поле для ввода идентификатора проекта
project_id_text = widgets.Textarea(
    value='',
    placeholder='Введите идентификатор проекта/страны (если несколько, то через запятую)',
    description='ID:',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='600px')  # Увеличиваем ширину текстового поля
)

# Функция для активации/деактивации текстового поля в зависимости от выбранной опции
def toggle_project_id_field(change):
    if project_options.value in [
        'Не указывать проект'
    ]:
        project_id_text.disabled = True
        project_id_text.value = ''
    else:
        project_id_text.disabled = False

# Привязка функции к изменениям в RadioButtons
project_options.observe(toggle_project_id_field, names='value')

# Упаковываем виджеты в вертикальный контейнер VBox
projects_data = widgets.VBox([project_options, project_id_text])

# Функция для получения выбранной опции и идентификатора проекта
def get_search_parameters():
    return {
        'selected_option': project_options.value,
        'project_id': project_id_text.value if project_id_text.value else None
    }

start = widgets.DatePicker(description='Начало периода',style=dict(description_width='100px'),
    disabled=False)

end = widgets.DatePicker(description='Конец периода',style=dict(description_width='100px'),
    disabled=False)


not_allowed = widgets.Checkbox(
    value=True,
    description='Добавить стандартные ограничения на пуши/рассылки',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

In [ ]:
grid1 = widgets.GridspecLayout(1,1, grid_gap='1px', width='650x')
grid1[0,0] =search_data
grid1

**Период, указанный вами, влияет на поиск пользователей в зависимости от даты его последнего запуска. **

Н-р, если вы поставите период с 01.07.2024 по 31.07.2024, то в выгрузке будут пользователи, у которых последний запуск приложения был в этот период

In [ ]:
grid2 = widgets.GridspecLayout(3,1, grid_gap='1px', width='650x')
grid2[0,0] =start
grid2[1,0] =end
grid2[2,0] =not_allowed
grid2

In [ ]:
grid3 = widgets.GridspecLayout(1,1, grid_gap='1px', width='650x')
grid3[0,0] = projects_data
grid3

In [ ]:
layout = widgets.Layout(width='auto', height='40px') #set width and height        

button = widgets.Button(description='Начать поиск',button_style='primary',layout = layout)
button.style.font_weight = 'bold'
button.on_click(main)

grid4 = widgets.GridspecLayout(1,1, grid_gap='3px', width='650x')
grid4[0,0] = button
grid4

In [ ]:
out = widgets.Output()
display(out)

In [ ]:
link_out = widgets.Output()
display(link_out)